In [6]:
%matplotlib inline
import pandas as pd
import datetime
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.utils.data.dataloader import default_collate
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class IMUDataset(Dataset):
    def __init__(self, csv_file, transform = None):
        self.df = pd.read_csv(csv_file, header = 0, index_col = 0)
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        y = self.df.iloc[idx : idx + 60, 3 : ].values
        ind = np.argmax(np.sum(y, axis = 0))
        label = np.zeros_like(self.df.iloc[0, 3 : ].values)
        label = label.astype(float)
        label[ind] = 1
        x = self.df.iloc[idx : idx + 60, : 3].values
        x = x.astype(float)
        assert(x.shape == (60, 3))
        assert(label.shape == (6, ))
        return x, label
        
dataset = IMUDataset(csv_file = 'data_preparation/final_data.csv')

In [11]:
def my_collate(batch):
    "Puts each data field into a tensor with outer dimension batch size"
    batch = list(filter(lambda x : x is not None, batch))
    return default_collate(batch)

indices = [(i * 60) for i in range(len(dataset) // 60)]
dataloader = DataLoader(dataset, batch_size = 16, sampler = SubsetRandomSampler(indices), collate_fn = my_collate, drop_last = True)
# for idx, (x, y) in enumerate(dataloader) :
#     print(x.shape, y.shape)

1020


In [14]:
class ConvAutoEncoder(nn.Module):
    def __init__(self, code_size):
        super(ConvAutoEncoder, self).__init__()
        self.code_size = code_size
        # defining layers
        # input size - batch_size x 3 x 100
        conv1 = nn.Conv1d(3, 10, 3)
        # after conv1 - batch_size x 10 x 98
        conv2 = nn.Conv1d(10, 20, 1)
        # after conv2 - batch_size x 10 x 98
        enc_fc = nn.Linear(10 * 98, self.code_size)
        
        dec_fc = nn.Linear(self.code_size, 10 * 98)
        deconv2 = nn.ConvTranspose1d(20, 10, 1)
        deconv1 = nn.ConvTranspose1d(10, 3, 3)
        
    def forward(self, signal):
        code = self.encode(signal)
        out = self.decode(code)
        return out, code
    
    def encode(signal):
        signal = signal.view(16, 3, 60)
        code = F.sigmoid(self.conv1(signal))
        code = F.sigmoid(self.conv2(code))
        code = F.sigmoid(self.enc_fc(code))
        return code
    
    def decode(code):
        out = F.sigmoid(self.dec_fc(code))
        out = F.sigmoid(self.deconv2(out))
        out = F.sigmoid(self.deconv1(out))
        return out

In [13]:
def output_size(n, f, p = 0, s = 1):
    return (((n + 2 * p - f) / s) + 1)

print(output_size(98, 1))

98.0
